In [14]:
import PIL
from PIL import Image
import matplotlib

import numpy as np
import pandas as pd

import ast
import re

### **Load Images**

Currently I saved all the images and their information in a csv, which means I don't need to open lots of images when I run my code, but it also means I have to decode the strings in the csv to get my images and labels

In [27]:
train_data = pd.read_csv("C:\\Users\\KurtJi\\OneDrive - University of Illinois - Urbana\\Desktop\\Personal Projects\\data\\inaturalist\\data_csv\\train_dataset.csv")

In [28]:
print(train_data.iloc[0,0])

C:\Users\KurtJi\OneDrive - University of Illinois - Urbana\Desktop\Personal Projects\data\inaturalist/train_mini/03198_Animalia_Chordata_Aves_Anseriformes_Anatidae_Anas_fulvigula/e706c2be-c819-4c81-bb34-d1f895eb7dec.jpg


In [12]:
ast.literal_eval(train_data.iloc[0,0])

SyntaxError: invalid syntax (<unknown>, line 1)

In [23]:
# Add spaces between numbers using regular expressions
array_string_spaces = re.sub(r'\s+(\d+)', r'\1', array_string_spaces)

In [24]:
array_string_spaces

'[[[65,63,24]\n  [88,86,45]\n  [147,146,98]\n  ...\n  [59,53,17]\n  [83,77,41]\n  [99,93,57]]\n\n [[67,65,26]\n  [90,89,45]\n  [150,149,101]\n  ...\n  [79,73,37]\n  [75,69,33]\n  [87,81,45]]\n\n [[54,51,10]\n  [81,78,35]\n  [146,144,96]\n  ...\n  [91,87,52]\n  [63,59,24]\n  [69,65,30]]\n\n ...\n\n [[223,223,221]\n  [219,219,217]\n  [213,213,211]\n  ...\n  [210,210,208]\n  [211,211,209]\n  [209,209,207]]\n\n [[225,226,221]\n  [225,226,221]\n  [227,228,223]\n  ...\n  [209,210,205]\n  [211,212,207]\n  [208,209,204]]\n\n [[201,202,197]\n  [200,201,196]\n  [198,199,194]\n  ...\n  [174,175,170]\n  [177,178,173]\n  [173,174,169]]]'

## **Image Processing**

### **Denoising using fft**

UnidentifiedImageError: cannot identify image file 'C:\\Users\\KurtJi\\OneDrive - University of Illinois - Urbana\\Desktop\\Personal Projects\\Image\\data\\test_data\\0.npy'

ValueError: Object arrays cannot be loaded when allow_pickle=False

array({'image': array([[[ 32,  44,  24],
        [ 31,  43,  23],
        [ 28,  40,  20],
        ...,
        [ 82,  99,  55],
        [ 81,  98,  56],
        [ 78,  96,  56]],

       [[ 34,  46,  26],
        [ 32,  44,  24],
        [ 29,  41,  21],
        ...,
        [ 75,  91,  46],
        [ 71,  88,  44],
        [ 71,  87,  48]],

       [[ 35,  47,  25],
        [ 34,  46,  24],
        [ 33,  45,  25],
        ...,
        [ 71,  87,  42],
        [ 65,  80,  37],
        [ 63,  78,  39]],

       ...,

       [[ 36,  11,   0],
        [ 43,  15,   0],
        [ 68,  32,   0],
        ...,
        [108, 106,  47],
        [ 93,  91,  34],
        [ 84,  81,  26]],

       [[ 43,  18,   0],
        [ 55,  28,   1],
        [ 85,  49,  15],
        ...,
        [ 91,  88,  37],
        [ 78,  74,  26],
        [ 71,  67,  20]],

       [[ 45,  20,   0],
        [ 71,  44,  17],
        [106,  70,  36],
        ...,
        [ 78,  74,  27],
        [ 73,  69,  24],
        